In [35]:
def build_toy_dataset(n_data, noise_std, split_size, seed, state):
    
    rs = npr.RandomState(state)
    inputs  = np.concatenate([np.linspace(0, 2, num=n_data//2),
                              np.linspace(6, 8, num=n_data//2)])
    targets = np.cos(inputs)*inputs + rs.randn(n_data) * noise_std
    inputs  = inputs.reshape((len(inputs), 1))
    targets = targets.reshape((len(targets), 1))
    
    train_count = int (split_size * n_data)
    idx = rs.permutation(range(n_data))
    inputs_train = inputs[idx[train_count:]]
    inputs_test = inputs[idx[:train_count]]
    targets_train = targets[ idx[train_count:], ]
    targets_test = targets[ idx[:train_count], ]
    
    return inputs_train, inputs_test, targets_train, targets_test, 1, 1., 0.

def build_boston_dataset(split_size,scale,state):
    
    boston = load_boston()
    bos = pd.DataFrame(boston.data)
    bos.columns = boston.feature_names
    if scale == True:
        a = bos.drop('CHAS',axis =1 )
        a=(a-np.mean(a))/np.std(a)
        a['CHAS']=bos['CHAS'] 
        
    else:
        a = bos
        train_std = 1.
        train_mu = 0.
    del bos
    
#     rs = npr.RandomState(state)
#     n_data = a.shape[0]
#     train_count = int (split_size * n_data)
#     idx = rs.permutation(range(n_data))
#     inputs_train = a[idx[train_count:],:]
#     inputs_test = a[idx[:train_count],:]
#     targets_train = boston.targets[idx[train_count:]]
#     targets_test = boston.targets[idx[:train_count]]
    inputs, inputs_test, targets,  targets_test = cv(a, boston.target, test_size = split_size, random_state = state)
    
    train_std = np.std(targets)
    train_mu = np.mean(targets)
    targets = (targets-train_mu) / train_std
    
    return inputs, inputs_test, np.expand_dims(targets,1),  np.expand_dims(targets_test,1), a.shape[1], train_std, train_mu

In [ ]:
def read_yacht(split_size,state):
    fx = open('data.txt', 'r')
    x = []
    y = []
    for line in fx:
        line = line.rsplit()
        x.append(line[:6])
        y.append(line[6])

    x = np.array(x, dtype=float)
    y = np.array(y, dtype=float)
    fx.close()
    inputs, inputs_test, targets,  targets_test = cv(x, y, test_size = split_size, random_state = state)
    return inputs, inputs_test, np.expand_dims(targets,1),  np.expand_dims(targets_test,1), x.shape[1], train_std, test_std

In [ ]:
def acc_rmse(y_true, y_pred, y_train_std, y_train_mu):
    y_pred = y_train_std * y_pred + y_train_mu
    rmse = np.sqrt(np.mean((y_true-y_pred) ** 2))
    return rmse